<a href="https://colab.research.google.com/github/shriyan44/Lab-6-DATA-301/blob/main/Copy_of_DATA_301_Lab_6B_Shriya_Nimmagadda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bing Maps API

In this part of the lab, you will join the CityBikes data from the previous part of the lab with additional data that you will query from the Bing Maps API.

First, you will need to register for a Bing Maps Key. Follow the instructions [here](https://docs.microsoft.com/en-us/bingmaps/getting-started/bing-maps-dev-center-help/getting-a-bing-maps-key). You should be able to sign in with your Cal Poly account. When you get to Step 4, select:

- Key type: Basic
- Application type: Dev/Test

You will be able to make 125000 free requests with the resulting API key. This should be more than enough to complete this assignment.

We will be working with the [REST services in the Bing Maps API](https://docs.microsoft.com/en-us/bingmaps/rest-services/). Click on the link for a complete documentation of the features.

## Question 1

Read in the `DataFrame` of bike stations in the United States from Part A of this lab. Restrict to the stations in the "Bay Wheels" network (with network ID "ford-gobike").

How many of these stations are in the city/county of San Francisco? 

(_Hint:_ Use the [Locations API](https://docs.microsoft.com/en-us/bingmaps/rest-services/locations/) to get the address associated with each latitude and longitude coordinate.)

In [11]:

# AjDIyFMNcKFfc6oWn8OmF_gxlT8rUMkepcqE6YB2z2EsGyyRFQMSAMXVoKNFH8Lq
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
df_stations = pd.read_csv("/content/drive/My Drive/stations.csv")
df_bay = df_stations[df_stations["networkid"] == "bay-wheels"]
# df_bay.reset_index(drop=True)
df_bay


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Unnamed: 0  empty_slots  free_bikes  ... extra.slots  networkname   networkid
6231        6231          8.0           7  ...         NaN   Bay Wheels  bay-wheels
6232        6232          3.0           8  ...         NaN   Bay Wheels  bay-wheels
6233        6233          3.0          20  ...         NaN   Bay Wheels  bay-wheels
6234        6234          2.0          13  ...         NaN   Bay Wheels  bay-wheels
6235        6235         23.0           3  ...         NaN   Bay Wheels  bay-wheels
...          ...          ...         ...  ...         ...          ...         ...
6708        6708         18.0           5  ...         NaN   Bay Wheels  bay-wheels
6709        6709         15.0          12  ...         NaN   Bay Wheels  bay-wheels
6710        6710          6.0           9  ...         NaN   Bay Wheels  bay-wheels
6711        6711          5.0          14  ...         NaN   Bay Wheels  bay-wheels
6712        6712         14.0           9  ...         NaN   Bay Wheels  bay-wheels

[482 rows x 20 columns]

In [2]:
df_bay1 = df_bay.loc[[6232,6234]]
df_bay1

Unnamed: 0  empty_slots  free_bikes  ... extra.slots  networkname   networkid
6232        6232         14.0           1  ...         NaN   Bay Wheels  bay-wheels
6234        6234          3.0          20  ...         NaN   Bay Wheels  bay-wheels

[2 rows x 20 columns]

In [29]:
import requests
import time
import pandas as pd
locinfo = []

for i in df_bay.index:
  response = requests.get("http://dev.virtualearth.net/REST/v1/Locations/" + str(df_bay["latitude"][i]) 
  + "," + str(df_bay["longitude"][i]) + "?includeEntityTypes=address&key=AjDIyFMNcKFfc6oWn8OmF_gxlT8rUMkepcqE6YB2z2EsGyyRFQMSAMXVoKNFH8Lq")
  locinfo.append(response.json()["resourceSets"][0])
  # print(locinfo)
  # print(i)
  time.sleep(0.2)



# df_sf = df_all[df_all["San Francisco"]]
# df_sf





In [ ]:
df_all = pd.json_normalize(locinfo, "resources", errors="ignore")
df_all["stationName"] = df_bay["name"].to_numpy()
df_all.head()


In [40]:
df_sf = df_all[df_all["address.adminDistrict2"] == "San Francisco Co."]
df_sf


__type  ...                                        stationName
25   Location:http://schemas.microsoft.com/search/l...  ...                              17th St at Dolores St
26   Location:http://schemas.microsoft.com/search/l...  ...                                  4th St at 16th St
27   Location:http://schemas.microsoft.com/search/l...  ...                        Scott St at Golden Gate Ave
28   Location:http://schemas.microsoft.com/search/l...  ...                                Folsom St at 3rd St
30   Location:http://schemas.microsoft.com/search/l...  ...                      Gennessee St at Monterey Blvd
..                                                 ...  ...                                                ...
477  Location:http://schemas.microsoft.com/search/l...  ...  Balboa Park (San Jose Ave at Sgt. John V. Youn...
478  Location:http://schemas.microsoft.com/search/l...  ...                  SFSU University Park North - West
479  Location:http://schemas.microsoft.com/search/l...  ...                  SFSU University Park North - East
480  Location:http://schemas.microsoft.com/search/l...  ...                          Brighton Ave at Ocean Ave
481  Location:http://schemas.microsoft.com/search/l...  ...                         Alemany Blvd at Francis St

[271 rows x 22 columns]

There are 271 SF stations out of 482 stations in the bay-wheels network in US

## Question 2

You want to go to Coit Tower. To save money, you decide to ride a "Bay Wheels" bike to the closest station and hail a cab from there to Coit Tower. What station should you bike to so that you are as close to Coit Tower as possible (as measured by driving distance)? Does your answer agree with the one that you obtained in Part A of this lab? If not, why does it differ?

_Hints:_ 
- You should restrict your attention to bike stations that are in San Francisco, which you determined in Question 1. 
- Use the [Routes API](https://docs.microsoft.com/en-us/bingmaps/rest-services/routes/calculate-a-distance-matrix) to calculate a distance matrix between Coit Tower and the bike stations. 
- You can do this with just one call to the API. Because there are too many stations, it is impossible to specify all the locations in the URL. Instead, you should make a POST request (`requests.post`), passing in the parameters as a JSON object through the `json=` parameter of `requests.post`. Read the API documentation carefully to learn how to use the POST API.

In [4]:
# 37.802747 and longitude -122.405861
def build_request_body():
  request_body = {
      "origins": [],
      "destinations": [{
        "latitude": 37.8024,
        "longitude": -122.4058
      }],
      "travelMode": "Driving"
  }
  for row_series in df_sf_bike_share.iterrows():
    request_body["origins"].append({
        "latitude": row_series[1]["latitude"],
        "longitude": row_series[1]["longitude"]
    })

  return request_body

response = requests.post("https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix?key=AgEzxjqI_hrzgJtK3P0q4dlzWf8aILYsp_uW3OEp9miAOUr7eNyXVFILjBBJJz2_", json=build_request_body())
df_distance_bike_share = pd.json_normalize(response.json()["resourceSets"][0]["resources"][0]["results"])
df_distance_bike_share = df_distance_bike_share.join(df_sf_bike_share.reset_index())

min_index = df_distance_bike_share["travelDistance"].idxmin()
df_distance_bike_share.loc[min_index]

## Submission Instructions

- Copy this notebook to your own Drive, if you have not already.
- Restart this notebook and run the cells from beginning to end. 
  - Go to Runtime > Restart and Run All.
- Rename this notebook by clicking on "DATA 301 Lab 6B - YOUR NAMES HERE" at the very top of this page. Replace "YOUR NAMES HERE" with the first and last names of you (and your partners, for Phase 2).
- Get the link to your notebook:
  - Click on "Share" at the top-right. 
  - Change the settings to "Anyone with the link can view". 
  - Copy the sharing link into Canvas.